# Imports y funciones

In [1]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point, LineString
from ipyleaflet import Map, Heatmap, GeoJSON
import geopy.distance
import xgboost as xgb
import googlemaps
import datetime
def nombre_columnas(x):
    out = sin_acento(x).lower().strip().replace(' ', '_').replace('\n', '').replace(':', '')
    return out
def sin_acento(x):
    output = x.replace('á','a').replace('é','e').replace('í','i').replace('ó','o').replace('ú','u')\
            .replace('Á','A').replace('É','E').replace('Í','I').replace('Ó','O').replace('Ú','U')
    return output
# Calcular un intervalo espacio-tiempo de los últimos viajes
def vel_reciente(i):
    """Calcula velocidad promedio de los viajes en las estaciones subsiguientes que van en el mismo sentido
    durante la última media hora"""
    interv1 = i['fecha'] 
    interv0 = interv1 - pd.Timedelta(30, unit='m')
    mytripid = i.name
    myest = i['est']
    mysigno = '>=' if i['sentido']==1 else '<='
    mysent = i['sentido']
    out = mb.query(f'"{interv0}"<=fecha<="{interv1}" & trip_id!="{mytripid}" & est{mysigno}{myest} & sentido=={mysent}')['velocidad']\
        .replace([pd.np.inf, -pd.np.inf], pd.np.nan)\
        .mean()
    return out
def n_estaciones(i):
    if i[['y1', 'y2']].notnull().all():
        li, ls = i[['y1', 'y2']].sort_values().tolist()
        out = len(estaciones_mb.query(f'{li}<=lat<={ls}'))
    else:
        out = pd.np.nan
    return out
def trafico(i):
    t_segm = i['t_segm']
    li, lu = sorted([i['est'], i['est_destino']])
    trafico = t_maps.query(f'est_origen>={li} & est_destino<={lu} & t_segm=={t_segm}').eval('duracion_en_trafico-duracion', inplace=False).sum()
    return trafico
def calcula_intersecciones(est_o, est_d):
    """Regresa el número de intersecciones viales que hay que cruzar para llegar de est1 a est2"""
    q = f'{est_o}<=est<={est_d}' if est_d > est_o else f'{est_d}<=est<={est_o}'
    df = breaks.query(q).unary_union
    out = vias.intersects(df).sum()
    return out
%matplotlib inline

# Rutas Ecobus

In [26]:
df = pd.read_csv('sm1_bicentenario.csv', sep=';')\
        .rename(columns=nombre_columnas)\
        .rename(columns={'velocidad_media_(km/h)': 'velocidad_media', 'ubicacion_de_parada': 'parada',
                        'fecha_de_inicio': 'fecha', 'hora_de_parada': 'hora'})\
        .sort_values(['id_unidad', 'fecha', 'hora'])\
        .assign(geometry=lambda x: x['punto_geografico'].apply(lambda y: Point(eval(y)) if isinstance(y, str) else pd.np.nan)) \
        .dropna(subset=['geometry'])\
        .pipe(lambda x: gpd.GeoDataFrame(x, geometry='geometry'))
circuito = df.cx[19.35:19.47, -99.20:-99.08].query('territorio=="R-BICENTENARIO"')
df2 = pd.read_csv('sm1_maquevedo_stafe.csv', sep=';')\
        .rename(columns=nombre_columnas)\
        .rename(columns={'velocidad_media_(km/h)': 'velocidad_media', 'ubicacion_de_parada': 'parada',
                        'fecha_de_inicio': 'fecha', 'hora_de_parada': 'hora'})\
        .sort_values(['id_unidad', 'fecha', 'hora'])\
        .assign(geometry=lambda x: x['punto_geografico'].apply(lambda y: Point(eval(y)) if isinstance(y, str) else pd.np.nan)) \
        .dropna(subset=['geometry'])\
        .pipe(lambda x: gpd.GeoDataFrame(x, geometry='geometry'))\
        .cx[19.25:19.45, -99.3:-99.1]

# Líneas Metrobus

In [101]:
lineas_mb = gpd.read_file('datos/metrobus-rutas.geojson')
bici = gpd.read_file('datos/estaciones_ecobici.geojson')
with open('datos/metrobus-rutas.geojson') as f, open('datos/estaciones_ecobici.geojson') as f_bici:
    geo_mb = GeoJSON(data=json.load(f))
    geo_bici = GeoJSON(data=json.load(f_bici))
locations = bici.apply(lambda x: (x['location_lat'], x['location_lon']), axis=1).tolist()

In [5]:
m = Map(center=(19.4, -99.17), zoom=11)
heat = Heatmap(locations=locations)
m.add_layer(geo_mb)
#m.add_layer(geo_bici)
m.add_layer(heat)
m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map data (c) <a href="https://openstreetmap.org">OpenStreetMap</a> contributors'}, center=[19.4, -99.17], layers=(TileLayer(base=True, max_zoom=19, min_zoom=1, options=['attribution', 'detect_retina', 'max_zoom', 'min_zoom', 'tile_size']), GeoJSON(data={'type': 'FeatureCollection', 'features': [{'type': 'Feature', 'geometry': {'type': 'LineString', 'coordinates': [[-99.17006270269148, 19.52944600000325], [-99.17013727993948, 19.52902696192144], [-99.170155245716, 19.52882856405869], [-99.1701450339526, 19.52869039965773], [-99.17011678155119, 19.52855207931649], [-99.17004988105641, 19.52840896135667], [-99.1699161612255, 19.52822536182644], [-99.16872087308882, 19.52673061782625], [-99.16800167921953, 19.52580753557234], [-99.16657905917245, 19.52390897206767], [-99.1663954508107, 19.52368447878274], [-99.16612525067663, 19.52338707147429], [-99.16535097684532, 19.52247376176771], [-99.16518758954477, 19.52228290968657], [-99.16499526428265, 19.52201237286918], [-99.1647853939285, 19.52167337594446], [-99.16447753361236, 19.52119532222231], [-99.16419925900334, 19.52058541138587], [-99.16374756802334, 19.51957155326732], [-99.16338400632424, 19.51880682860313], [-99.16310778269441, 19.51832222518099], [-99.16281525072428, 19.51770866579599], [-99.1617138059061, 19.51488530892275], [-99.16053217286866, 19.51194105397645], [-99.1593033109265, 19.5087502920225], [-99.15793390311292, 19.50520485487802], [-99.15750290818914, 19.50419917050762], [-99.15672826378882, 19.50232258792643], [-99.155717418006, 19.49945241417748], [-99.15416028907745, 19.49520288358871], [-99.15244890570105, 19.49079381779069], [-99.1509422259235, 19.48683739951879], [-99.14928641101916, 19.48230603444944], [-99.14755313153988, 19.47778542739743], [-99.1459524560623, 19.47350230576858], [-99.14430306750423, 19.46918310519536], [-99.14374949728582, 19.46766411380752], [-99.14350291242603, 19.46702407404441], [-99.14325278048759, 19.46632118196732], [-99.1430750584207, 19.46578695280278], [-99.14298789643983, 19.46543971080493], [-99.143003633282, 19.46524892046701], [-99.14308647306032, 19.46493260439379], [-99.14323323806487, 19.46461997676112], [-99.1433735795718, 19.46429434868447], [-99.14370851933919, 19.46381301644861], [-99.14426031011955, 19.46326309268968], [-99.14460320300702, 19.46286949476769], [-99.14475608016133, 19.46259958887917], [-99.14479021659119, 19.46243384464469], [-99.14477046577308, 19.46224400327788], [-99.14472371124688, 19.46205525821499], [-99.14456562232243, 19.46180673459119], [-99.14429915954403, 19.46154564776894], [-99.14406586181013, 19.46129812539308], [-99.14396059796509, 19.46111193598489], [-99.14391219075024, 19.46092628237353], [-99.14390375219303, 19.4607499066719], [-99.1439324812973, 19.46053880211295], [-99.14435127397589, 19.45867316560257], [-99.14505949067525, 19.45559019411321], [-99.14590451320646, 19.45187695613705], [-99.14674432084848, 19.44817050292471], [-99.14761519988828, 19.44432444240548], [-99.14818180752981, 19.44182057932355], [-99.14836686235945, 19.44112735521211], [-99.14861037881555, 19.44034642483874], [-99.14873430914041, 19.43993392069117], [-99.14880790503256, 19.43955354645772], [-99.14891304282634, 19.43869662608034], [-99.14898691134425, 19.43822729791832], [-99.14900992923954, 19.43799418707076], [-99.14894709608417, 19.43788267416638], [-99.14872178228933, 19.43780706917011], [-99.14751138001701, 19.43757582843707], [-99.14715019872745, 19.43750332698388], [-99.14701681103541, 19.43741950104495], [-99.1469605726163, 19.43730226000798], [-99.14694644645485, 19.43710623503026], [-99.14704883860583, 19.43653815474976], [-99.14728943079938, 19.43504312052377], [-99.14744815265195, 19.43423409959562], [-99.14779522744924, 19.43254009118917], [-99.14823217661173, 19.43040127938817], [-99.14855549741269, 19.42891027103345], [-99.148801033141, 19.4275933012513], [-99.14887582054806, 19.

# Rutas MetroBus

## Estaciones mb

In [343]:
linea1 = gpd.read_file('datos/metrobus-rutas.geojson').query('name=="1 - Insurgentes"').iloc[0,-1]
estaciones_mb = pd.read_csv('datos/estaciones-metrobus.csv', sep=';')\
    .assign(coord0=lambda x: x['wkt_geom'].str.extract(r'(\-\d+\.\d+ \d+\.\d+)'),
           lat = lambda x: x['wkt_geom'].str.extract(r'( \d+\.\d+)'))\
    .assign(coord=lambda x: x['coord0'].apply(lambda y: tuple(map(float, y.split()))))\
    .assign(geometry=lambda x: x['coord'].apply(Point))\
    .query('Linea=="Línea 1"')\
    .astype({'lat': float})\
    .sort_values('lat', ascending=False)\
    .pipe(lambda df: gpd.GeoDataFrame(df, geometry='geometry'))\
    .assign(est=lambda x: x['lat'].rank(ascending=False).astype(int)-1)
df_estaciones_mb = pd.DataFrame(estaciones_mb)
df_estaciones_mb.to_pickle('datos/estaciones_mb.pkl')

## Segmento de ruta

In [356]:
## Cortamos en 47 segmentos la línea 1
puntos_linea = pd.DataFrame({'coord': list(linea1.coords)})\
    .assign(geometry=lambda x: x['coord'].apply(Point))\
    .pipe(lambda x: gpd.GeoDataFrame(x, geometry='geometry'))
breaks = gpd.GeoDataFrame.from_records(
        [{'est': i,
          'nom_est': estaciones_mb.iloc[i]['Nombre'],
          'geometry': LineString([puntos_linea.loc[puntos_linea.distance(estaciones_mb.iloc[i]['geometry']).idxmin(), 'geometry'],
                    puntos_linea.loc[puntos_linea.distance(estaciones_mb.iloc[i+1]['geometry']).idxmin(), 'geometry']])}
         for i in range(0, len(estaciones_mb)-1)])\
        .sort_values('est')
breaks.to_pickle('datos/breaks.pkl')

## Intersecciones viales con L1

In [357]:
vias_df = gpd.read_file('datos/vialidades_df/vialidades_df.geojson', encoding='utf-8')
vias = vias_df.loc[vias_df.intersects(linea1)]\
            .pipe(lambda x: x.loc[~x['NOMVIAL'].str.contains('INSURGENTES')])
intersecciones = pd.DataFrame(
    [{'est_o': i, 'est_d': i+1, 'intersecciones':calcula_intersecciones(i, i+1)}
     for i in range(0, len(estaciones_mb)-1)])
intersecciones.to_pickle('datos/intersecciones.pkl')

## Clima

In [326]:
clima = pd.read_csv('datos/clima.csv')\
    .query('Day==14')\
    .rename(columns={'Hour': 'hora'})\
    [['hora', 'precip']]
clima.to_pickle('datos/clima.pkl')

## Tiempos gmaps

In [6]:
t_maps = pd.read_pickle('datos/t_gmaps_mb_l1.pkl')

## Datos viajes

In [335]:
mb = pd.read_csv('datos/metrobus_linea1.csv', sep=';', dtype={'trip_id': 'str'})\
        .dropna(subset=['trip_id'])\
        .assign(coord=lambda x: list(zip(x['position_longitude'], x['position_latitude'])))\
        .assign(geometry=lambda x: x['coord'].apply(Point),
               fecha=lambda x: pd.to_datetime(x['date_updated']))\
        .sort_values(['trip_id', 'fecha'])\
        .assign(t_diff=lambda x: x.groupby('trip_id')['fecha'].diff(),
               t_segm =lambda x: 6 * x['fecha'].dt.hour + x['fecha'].dt.minute// 10,
               hora = lambda x: x['fecha'].dt.hour)\
        .dropna(subset=['geometry'])\
        .pipe(lambda x: gpd.GeoDataFrame(x, geometry='geometry'))\
        .cx[-99.1:-99.3, 19:19.6]\
        .set_index('trip_id')\
        .assign(y1=lambda x: x['position_latitude'].groupby(level=0).shift(1),
                y2=lambda x: x['position_latitude'],
                x1=lambda x: x['position_longitude'].groupby(level=0).shift(1),
                x2=lambda x: x['position_longitude'])\
        .assign(ruta=lambda mb: mb.apply(lambda x: LineString([(x['x1'], x['y1']), (x['x2'], x['y2'])])
                                         if x[['x1', 'y1', 'x2', 'y2']].notnull().all() else pd.np.nan, axis=1),
               distancia=lambda mb: mb.apply(lambda x: geopy.distance.distance((x['y1'], x['x1']), (x['y2'], x['x2'])).km
                           if x[['x1', 'y1', 'x2', 'y2']].notnull().all() else pd.np.nan, axis=1),
                sentido=lambda mb: (mb['y1']-mb['y2']).apply(lambda x: 1 if x<=0 else -1),
                est=lambda x: x['geometry'].apply(lambda i: breaks.loc[breaks.distance(i).idxmin(), 'est']),
               tiempo_s = lambda x: x['t_diff'].dt.total_seconds(),
               dist_l1 = lambda x: x.distance(linea1))\
        .assign(intersecciones=lambda mb: mb['ruta'].apply(lambda x: vias.intersects(x).sum() if not isinstance(x, float) else pd.np.nan))\
        .eval("""
        velocidad = distancia/(tiempo_s/3600)
        tiempo_m = tiempo_s/60
        """, inplace=False)\
        .assign(velocidad = lambda x: x['velocidad'].replace([pd.np.inf, -pd.np.inf], pd.np.nan))\
        .pipe(lambda x: x.loc[x['dist_l1']<x['dist_l1'].quantile(0.995)])\
        .reset_index()\
        .merge(clima, on='hora', how='left')\
        .dropna(subset=['tiempo_s', 'distancia'])\
        .query('tiempo_s!=0 & velocidad<=80 & distancia<=3')\
        .sort_values(['trip_id', 'fecha'])\
        .set_index('trip_id')
        
mb['reciente'] = mb.apply(vel_reciente, axis=1)
mb['n_estaciones'] = mb.apply(n_estaciones, axis=1)
mb['est_destino'] = (mb['est']+ mb['sentido']*mb['n_estaciones']).clip(0, 46)
mb['trafico'] = mb.apply(trafico, axis=1)
filename = 'datos/datos_modelo_mb_l1.pkl'
mb.to_pickle(filename)
dataframe = pd.DataFrame(mb)
dataframe.to_pickle('datos/df_mb_l1.pkl')

# Modelo

## Importando datos

In [332]:
mb = pd.read_pickle('datos/df_mb_l1.pkl')
x_vars = ['distancia', 'est', 't_segm', 'sentido', 'intersecciones', 'reciente', 'n_estaciones', 'precip', 'trafico']
data = mb    
train_ind = pd.np.random.choice([False, True], size=len(data), p=[0.1, 0.9])
data_train = data.loc[train_ind]
x_train = data_train[x_vars]
ys_train = data_train['tiempo_s']
data_test = data.loc[~train_ind]
x_test = data_test[x_vars]
ys_test = data_test['tiempo_s']
xg_train = xgb.DMatrix(x_train, label=ys_train)
xg_test = xgb.DMatrix(x_test, label=ys_test)

## Entrenando modelo

In [334]:
param = {'max_depth': 3, 'eta': 0.8, 'silent': 1, 'objective': 'count:poisson',
        'nthread': 4, 'eval_metric': 'rmse'}
num_round = 120
evallist = [(xg_test, 'eval'), (xg_train, 'train')]
bst = xgb.train(param, xg_train, num_round, evallist)
bst.save_model('0001.model')

[0]	eval-rmse:84.2928	train-rmse:353.106
[1]	eval-rmse:83.7669	train-rmse:352.973
[2]	eval-rmse:82.8508	train-rmse:352.745
[3]	eval-rmse:81.2614	train-rmse:352.354
[4]	eval-rmse:78.5258	train-rmse:351.696
[5]	eval-rmse:73.898	train-rmse:350.626
[6]	eval-rmse:66.4038	train-rmse:349.008
[7]	eval-rmse:56.0202	train-rmse:346.979
[8]	eval-rmse:49.3335	train-rmse:345.603
[9]	eval-rmse:42.7436	train-rmse:344.084
[10]	eval-rmse:36.8088	train-rmse:342.673
[11]	eval-rmse:30.045	train-rmse:341.383
[12]	eval-rmse:28.9637	train-rmse:339.908
[13]	eval-rmse:28.5311	train-rmse:337.703
[14]	eval-rmse:28.4611	train-rmse:332.94
[15]	eval-rmse:31.3241	train-rmse:326.065
[16]	eval-rmse:28.9581	train-rmse:314.908
[17]	eval-rmse:29.3436	train-rmse:300.421
[18]	eval-rmse:28.6205	train-rmse:289.749
[19]	eval-rmse:28.2893	train-rmse:281.042
[20]	eval-rmse:27.3768	train-rmse:268.806
[21]	eval-rmse:29.7041	train-rmse:247.476
[22]	eval-rmse:28.9199	train-rmse:213.346
[23]	eval-rmse:30.0612	train-rmse:206.751
[24]	

## Predicciones

In [289]:
ypred = bst.predict(xg_test)
result = pd.DataFrame({'y': ys_test, 'pred': ypred})
rsme = pd.np.sqrt(result.eval('(y-pred)**2', inplace=False).mean())

# Crea predictores

In [385]:
t_maps = pd.read_pickle('datos/t_gmaps_mb_l1.pkl')
clima = pd.read_pickle('datos/clima.pkl')
mb = pd.read_pickle('datos/df_mb_l1.pkl')
estaciones_mb = pd.read_pickle('datos/estaciones_mb.pkl')
intersecciones = pd.read_pickle('datos/intersecciones.pkl')
modelo = xgb.Booster({'nthread': 4})
modelo.load_model('0001.model')

In [393]:
def get_t_estimado(est_o, est_d, h, m):
    time = pd.datetime(2018, 11, 14, h, m)
    sentido = get_sentido(est_o, est_d)
    pares = [[i, i+sentido*1] for i in range(est_o, est_d, sentido)]
    resultado = dict()
    for j in pares:
        datos_pred = pd.DataFrame([
                {'distancia': get_distancia(j[0], j[1]), 
                 'est': j[0],
                 't_segm': get_t_segm(time.hour, time.minute),
                 'sentido': sentido,
                 'intersecciones': get_intersecciones(j[0], j[1]),
                 'reciente': get_reciente(j[0], j[1], time.hour, time.minute),
                 'n_estaciones': get_n_estaciones(j[0], j[1]),
                 'precip': get_precip(time.hour),
                 'trafico': get_trafico(j[0], j[1], time.hour, time.minute)}]
                )[x_vars]
        x_test = xgb.DMatrix(datos_pred)
        t_pred = modelo.predict(x_test)[0]
        resultado[j[0]] = t_pred
        time += pd.Timedelta(t_pred, unit='s')
    out = pd.Series(resultado, name='tiempo')
    return out
def get_trafico(est_o, est_d, h, m):
    t_segm = get_t_segm(h, m)
    li, lu = sorted([est_o, est_d])
    trafico = t_maps.query(f'est_origen>={li} & est_destino<={lu} & t_segm=={t_segm}').eval('duracion_en_trafico-duracion', inplace=False).sum()
    return trafico
def get_precip(h):
    cl = clima.set_index('hora')
    out = cl.loc[h, 'precip']
    return out
def get_reciente(est_o, est_d, h, m):
    """Calcula velocidad promedio de los viajes en las estaciones subsiguientes que van en el mismo sentido
    durante la última media hora"""
    interv1 = max(pd.datetime(2018, 11, 14, 16, 46), pd.datetime(2018, 11, 14, h, m))
    interv0 = interv1 - pd.Timedelta(120, unit='m')
    myest = est_o
    mysigno = '>=' if get_sentido(est_o, est_d)==1 else '<='
    mysent = get_sentido(est_o, est_d)
    out = mb.query(f'"{interv0}"<=fecha<="{interv1}" & est{mysigno}{myest} & sentido=={mysent}')['velocidad']\
        .replace([pd.np.inf, -pd.np.inf], pd.np.nan)\
        .mean()
    return out
def get_n_estaciones(est_o, est_d):
    out = abs(est_o-est_d)
    return out
def get_sentido(est_o, est_d):
    out = 1 if est_d>est_o else -1
    return out
def get_t_segm(h, m):
    out = min(max(97, 6 * h + m//10), 143)
    return out
def get_distancia(est_o, est_d):
    """Calcula la distancia euclidiana para llegar de est1 a est 2"""
    df = estaciones_mb.set_index('est')
    dist = geopy.distance.distance(reversed(df.loc[est_o, 'coord']), reversed(df.loc[est_d, 'coord'])).km
    return dist
def get_intersecciones(est_o, est_d):
    est_o, est_d = sorted([est_o, est_d])
    out = intersecciones.query(f'est_o>={est_o} & est_d<={est_d}')['intersecciones'].sum()
    return out

In [394]:
r = get_t_estimado(0, 10, 19, 30)

In [411]:
r.clip(lower=30, upper=900)

0    63.101429
1    56.199306
2    64.117546
3    60.158165
4    57.066273
5    56.377716
6    61.642010
7    71.719200
8    61.136791
9    58.940643
Name: tiempo, dtype: float64

In [403]:
geopy.__version__

'1.16.0'